In [129]:
# Import the library.
# Import the library.
from googleads import ad_manager
from datetime import date,datetime
import tempfile
import gzip
import pandas as pd
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import config


In [130]:
# Initialize a client object, by default uses the credentials in ~/googleads.yaml.
client = ad_manager.AdManagerClient.LoadFromStorage()

# Initialize a service.
network_service = client.GetService('NetworkService', version='v202202')

# Make a request.
current_network = network_service.getCurrentNetwork()

print("Current network has network code '%s' and display name '%s'." %
      (current_network['networkCode'], current_network['displayName']))

Current network has network code '1009103' and display name 'NSTP e-Media Sdn Bhd'.


In [131]:
BQ_KEY_LOCATION = config.BQ_KEY_LOCATION
credentials = service_account.Credentials.from_service_account_file(BQ_KEY_LOCATION)

client_mpd = bigquery.Client(credentials = credentials, project = "mpdbq1", location = "US")

In [169]:
from googleads import errors

#def get_report():
    # statement = (ad_manager.StatementBuilder(version='v202202')
    # 			.Where('id = :id')
    # 			.WithBindVariable('id', int(10))
    # 			.Limit(None)
    # 			.Offset(None))

end_date = datetime.now().date()
start_date = date(date.today().year, 1, 1)

report_job = {
    'reportQuery': {
        'dimensions': ['WEEK', 'DEVICE_CATEGORY_NAME', 'DEVICE_CATEGORY_ID', "LINE_ITEM_TYPE",'LINE_ITEM_NAME','LINE_ITEM_ID', 'PARENT_AD_UNIT_ID',
                       'AD_UNIT_NAME', 'AD_UNIT_ID'],
        'columns': ['TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS', 'TOTAL_LINE_ITEM_LEVEL_CLICKS',
                    'TOTAL_LINE_ITEM_LEVEL_ALL_REVENUE'],
        'dateRangeType': 'CUSTOM_DATE',
        'startDate': start_date,
        'endDate': end_date
    }
}

report_downloader = client.GetDataDownloader(version='v202202')

try:
    report_job_id = report_downloader.WaitForReport(report_job)
except errors.AdManagerReportError as e:
    print(f'Failed to generate report. Error was: {e}')

export_format = 'CSV_DUMP'

report_file = tempfile.NamedTemporaryFile(suffix='.csv',delete=False)

report_downloader.DownloadReportToFile(
        report_job_id, export_format, report_file)

report_file.close()

print(f'Report job with id {report_job_id} downloaded to: \n{report_file.name}')

with open(report_file.name,'rb') as report:
        temp = gzip.GzipFile(fileobj=report)
        df = pd.read_csv(temp, sep=',')

Report job with id 12904252710 downloaded to: 
/var/folders/qd/zcfrhx7x6wv1klr64f0qnc7h0000gn/T/tmpq8f0csba.csv


In [171]:
df.columns, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467466 entries, 0 to 467465
Data columns (total 12 columns):
 #   Column                                    Non-Null Count   Dtype 
---  ------                                    --------------   ----- 
 0   Dimension.WEEK                            467466 non-null  object
 1   Dimension.DEVICE_CATEGORY_NAME            467466 non-null  object
 2   Dimension.DEVICE_CATEGORY_ID              467466 non-null  int64 
 3   Dimension.LINE_ITEM_TYPE                  467466 non-null  object
 4   Dimension.LINE_ITEM_NAME                  467466 non-null  object
 5   Dimension.LINE_ITEM_ID                    467466 non-null  int64 
 6   Dimension.PARENT_AD_UNIT_ID               467466 non-null  object
 7   Dimension.AD_UNIT_NAME                    467466 non-null  object
 8   Dimension.AD_UNIT_ID                      467466 non-null  int64 
 9   Column.TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS  467466 non-null  int64 
 10  Column.TOTAL_LINE_ITEM_LEVEL_CLI

(Index(['Dimension.WEEK', 'Dimension.DEVICE_CATEGORY_NAME',
        'Dimension.DEVICE_CATEGORY_ID', 'Dimension.LINE_ITEM_TYPE',
        'Dimension.LINE_ITEM_NAME', 'Dimension.LINE_ITEM_ID',
        'Dimension.PARENT_AD_UNIT_ID', 'Dimension.AD_UNIT_NAME',
        'Dimension.AD_UNIT_ID', 'Column.TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS',
        'Column.TOTAL_LINE_ITEM_LEVEL_CLICKS',
        'Column.TOTAL_LINE_ITEM_LEVEL_ALL_REVENUE'],
       dtype='object'),
 None)

In [159]:
# df= df.replace(',','')
df.replace(',','', regex=True, inplace=True)

In [161]:
parent_ad_unit_list=['183717733', '22235807415', '21676101156', '21872721896', '21872873235']

In [165]:
df = df[df['Dimension.LINE_ITEM_NAME'].str.contains('tonton', case=False, na=False) & df['Dimension.PARENT_AD_UNIT_ID'].isin(parent_ad_unit_list)]

In [176]:
df = df.assign(REVENUE = lambda x: round((x['Column.TOTAL_LINE_ITEM_LEVEL_ALL_REVENUE'] /1000000),2))

In [186]:
# df[['Column.TOTAL_LINE_ITEM_LEVEL_ALL_REVENUE','REVENUE']]
df.drop(['Column.TOTAL_LINE_ITEM_LEVEL_ALL_REVENUE'], axis=1, inplace=True)

In [184]:
df.rename(columns={"Dimension.WEEK":"WEEK","Dimension.DEVICE_CATEGORY_NAME":"DEVICE_CATEGORY_NAME","Dimension.DEVICE_CATEGORY_ID":"DEVICE_CATEGORY_ID", "Dimension.LINE_ITEM_TYPE":"LINE_ITEM_TYPE","Dimension.LINE_ITEM_NAME":"LINE_ITEM_NAME","Dimension.LINE_ITEM_ID":"LINE_ITEM_ID","Dimension.PARENT_AD_UNIT_ID":"PARENT_AD_UNIT_ID","Dimension.AD_UNIT_NAME":"AD_UNIT_NAME","Dimension.AD_UNIT_ID":"AD_UNIT_ID",
                   "Column.TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS":"TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS","Column.TOTAL_LINE_ITEM_LEVEL_CLICKS":"TOTAL_LINE_ITEM_LEVEL_CLICKS"}, inplace=True)

In [167]:
# df[df['AD_UNIT_ID']=='21764206526']
df.columns

,WEEK,DEVICE_CATEGORY_NAME,DEVICE_CATEGORY_ID,LINE_ITEM_TYPE,LINE_ITEM_NAME,LINE_ITEM_ID,PARENT_AD_UNIT_ID,AD_UNIT_NAME,AD_UNIT_ID,TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS,TOTAL_LINE_ITEM_LEVEL_CLICKS,TOTAL_LINE_ITEM_LEVEL_ALL_REVENUE
3184,2021-12-26/2022-01-01,Desktop,30000,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,183717733,Tonton_728x90,183717733,2062,0,1906485
3185,2021-12-26/2022-01-01,Desktop,30000,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,21676101156,Tonton_300x250,21676101156,174,0,126054
8245,2021-12-26/2022-01-01,Smartphone,30001,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,21676101156,Tonton_300x250,21676101156,4912,19,5805721
8248,2021-12-26/2022-01-01,Smartphone,30001,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,21872721896,Tonton_300x250_a,21872721896,3972,1,1559321
8249,2021-12-26/2022-01-01,Smartphone,30001,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,21872873235,Tonton_320x50,21872873235,4945,24,7248177
...,...,...,...,...,...,...,...,...,...,...,...,...
466703,2022-05-15/2022-05-21,Tablet,30002,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,21872873235,Tonton_320x50,21872873235,261,1,170747
467300,2022-05-15/2022-05-21,Connected TV,30004,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,183717733,Tonton_728x90,183717733,1838,1,351754
467301,2022-05-15/2022-05-21,Connected TV,30004,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,21676101156,Tonton_300x250,21676101156,300,0,36107
467303,2022-05-15/2022-05-21,Connected TV,30004,AD_EXCHANGE,Tonton Extra_Tonton_MGO,664145893,21872721896,Tonton_300x250_a,21872721896,141,0,11696


In [187]:
df.columns, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467466 entries, 0 to 467465
Data columns (total 12 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   WEEK                               467466 non-null  object 
 1   DEVICE_CATEGORY_NAME               467466 non-null  object 
 2   DEVICE_CATEGORY_ID                 467466 non-null  int64  
 3   LINE_ITEM_TYPE                     467466 non-null  object 
 4   LINE_ITEM_NAME                     467466 non-null  object 
 5   LINE_ITEM_ID                       467466 non-null  int64  
 6   PARENT_AD_UNIT_ID                  467466 non-null  object 
 7   AD_UNIT_NAME                       467466 non-null  object 
 8   AD_UNIT_ID                         467466 non-null  int64  
 9   TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS  467466 non-null  int64  
 10  TOTAL_LINE_ITEM_LEVEL_CLICKS       467466 non-null  int64  
 11  REVENUE                            4674

(Index(['WEEK', 'DEVICE_CATEGORY_NAME', 'DEVICE_CATEGORY_ID', 'LINE_ITEM_TYPE',
        'LINE_ITEM_NAME', 'LINE_ITEM_ID', 'PARENT_AD_UNIT_ID', 'AD_UNIT_NAME',
        'AD_UNIT_ID', 'TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS',
        'TOTAL_LINE_ITEM_LEVEL_CLICKS', 'REVENUE'],
       dtype='object'),
 None)

In [140]:
    #name of table and its schema
table_id = "admanager.Display_ads_Inventory_Device_Category_Week"
job_config = bigquery.LoadJobConfig()
job_config.schema = [
    bigquery.SchemaField("WEEK","STRING"),
    bigquery.SchemaField("DEVICE_CATEGORY_NAME","STRING"),
    bigquery.SchemaField("DEVICE_CATEGORY_ID","INTEGER"),
    bigquery.SchemaField("LINE_ITEM_NAME","STRING"),
    bigquery.SchemaField("LINE_ITEM_ID","INTEGER"),
    bigquery.SchemaField("PARENT_AD_UNIT_ID","STRING"),
    bigquery.SchemaField("AD_UNIT_NAME","STRING"),
    bigquery.SchemaField("AD_UNIT_ID","INTEGER"),
    bigquery.SchemaField("TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS","INTEGER"),
    bigquery.SchemaField("TOTAL_LINE_ITEM_LEVEL_CLICKS","INTEGER"),
    bigquery.SchemaField("REVENUE","FLOAT")]

In [141]:
job = client_mpd.load_table_from_dataframe(
df, table_id, job_config=job_config)
job.result()

LoadJob<project=mpdbq1, location=US, id=18cde2e9-7fc7-452f-acd4-c94817dc3579>

In [142]:
datetime.now()

datetime.datetime(2022, 5, 18, 13, 6, 57, 662881)